In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import bayesian_dpddm

ModuleNotFoundError: No module named 'models'